---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [5]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [6]:
def answer_one():
        
    data = pd.read_csv('Employee_Movie_Choices.txt',delimiter ='\t',names=['Employee','Movie'])
    data = data.drop(0).reset_index(drop = True)
    G = nx.Graph()
    G.add_nodes_from(data.Employee, bipartite = 0)
    G.add_nodes_from(data.Movie, bipartite = 1)
    for i in range(len(data.Employee)):
        G.add_edges_from([(data.loc[i].Employee,data.loc[i].Movie)])
    #plot_graph(G)
    #print(G.edges(data = True))
    return G
#answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [7]:
def answer_two():
    
    G = answer_one()
    for i in employees:
        G.node[i]['type'] = "employee"
    for i in movies:
        G.node[i]['type'] = "movie"
    #print (G.nodes(data=True))
    return G
#answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [19]:
def answer_three():
        
    G = answer_two()
    B = bipartite.weighted_projected_graph(G, employees)
    #edge_weights = {(u,v,):d['weight'] for u,v,d in B.edges(data=True)}
    
    
    return B
#answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [83]:
def answer_four():
        
    data = pd.read_csv('Employee_Relationships.txt',delimiter ='\t', names = ['E1', 'E2', 'RS'])
    G = answer_three()
    edge_weights = {(u,v,):d['weight'] for u,v,d in G.edges(data=True)}
    #print(data)
    df = pd.DataFrame(list(edge_weights), columns = ['E1', 'E2'])
    W = []
    for i in range(len(df)):
        W.append(G[df.loc[i].E1][df.loc[i].E2]['weight'])
    df['W'] = W
    temp1 = df['E1'].append(df.E2).reset_index(drop = True)
    temp2 = df['E2'].append(df.E1).reset_index(drop = True)
    temp3 = df['W'].append(df.W).reset_index(drop = True)
    final = pd.DataFrame()
    final['E1'] = temp1
    final['E2'] = temp2
    final['W'] = temp3
    final = final.sort_values(by=['E1']).reset_index(drop=True)
    ans = (pd.merge(data, final, on = ['E1', 'E2'], how = 'left'))
    ans['W'] = ans['W'].fillna(0)
    #print(ans)
    res = (ans.corr(method = 'pearson')['W']['RS'])
    return res# Your Answer Here
#answer_four()

0.78839622217334759